In [1]:
import torch
%matplotlib inline
import torch.nn as nn
import torch.optim as optim
import time
from sklearn.metrics import r2_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split



from google.colab import files
uploaded = files.upload()
data = pd.read_csv("Housing.csv")


features =['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

X = data[features].to_numpy(dtype=float)
y = data['price'].to_numpy(dtype=float)

X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y.reshape(-1, 1))

## Convert to pytorch sensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

X_train, X_val, y_train, y_val = train_test_split(
    X_tensor, y_tensor, test_size = 0.2, random_state = 23)

Saving Housing.csv to Housing.csv


In [4]:
## Neural network
model = nn.Sequential(
    nn.Linear(5,8), ## Input > Hidden Layer 1
    nn.ReLU(),
    nn.Linear(8,8), ## Hidden Layer 1 > Hidden Layer 2
    nn.ReLU(),
    nn.Linear(8,8), ## Hidden Layer 2 > Hidden Layer 3
    nn.ReLU(),
    nn.Linear(8,1) ## Output
)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

num_epochs = 200
train_losses = []
val_losses = []

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train)
    train_loss = loss_fn(y_pred, y_train)
    train_loss.backward()
    optimizer.step()

    ## Validation
    model.eval()
    with torch.no_grad():
        y_val_pred = model(X_val)
        val_loss = loss_fn(y_val_pred, y_val)

    train_losses.append(train_loss.item())
    val_losses.append(val_loss.item())

    if epoch % 20 == 0:
        print(f"Epoch {epoch:3d} | Train Loss: {train_loss.item():.4f} | Val Loss: {val_loss.item():.4f}")
print(f"Epoch {num_epochs} (Final) | Train Loss: {train_loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

training_time = time.time() - start_time

y_val_true = y_val.detach().numpy()
y_val_pred = y_val_pred.detach().numpy()
r2 = r2_score(y_val_true, y_val_pred)

print("\n Final Results")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Final Training Loss: {train_losses[-1]:.4f}")
print(f"Final Validation Loss: {val_losses[-1]:.4f}")
print(f"Validation R² Score: {r2:.4f}")

Epoch   0 | Train Loss: 1.1507 | Val Loss: 1.0018
Epoch  20 | Train Loss: 0.5151 | Val Loss: 0.4667
Epoch  40 | Train Loss: 0.4527 | Val Loss: 0.3967
Epoch  60 | Train Loss: 0.4268 | Val Loss: 0.3933
Epoch  80 | Train Loss: 0.4108 | Val Loss: 0.3943
Epoch 100 | Train Loss: 0.3957 | Val Loss: 0.4088
Epoch 120 | Train Loss: 0.3831 | Val Loss: 0.4160
Epoch 140 | Train Loss: 0.3599 | Val Loss: 0.4292
Epoch 160 | Train Loss: 0.3374 | Val Loss: 0.4426
Epoch 180 | Train Loss: 0.3255 | Val Loss: 0.4646
Epoch 200 (Final) | Train Loss: 0.3196 | Val Loss: 0.4652

 Final Results
Training Time: 0.23 seconds
Final Training Loss: 0.3196
Final Validation Loss: 0.4652
Validation R² Score: 0.4161
